In [ ]:
!pip install -qU transformers
!pip install -qU datasets
!pip install -qU seqeval
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 624.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import transformers
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForTokenClassification
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_ner = pd.read_csv ("/content/ner_datasetreferencefull.csv", encoding='latin1')
df_ner.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [ ]:
df_ner.shape

(1048364, 4)

In [ ]:
non_null_index = df_ner[~df_ner["Sentence #"].isnull()].index

#### Applying Sentence Name on every word whereever its null
for i in range(len(non_null_index) - 1):
    idx = non_null_index[i]
    next_idx = non_null_index[i+1]
    df_ner.iloc[idx + 1:next_idx].loc[:,"Sentence #"] = df_ner["Sentence #"].iloc[idx]


df_ner.head(30)



,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [ ]:
len(df_ner['Sentence #'].unique())

10201

### combining all the words

In [ ]:
#This process essentially consolidates the information within each "Sentence #" group into a single row, where the words, parts-of-speech, and tags belonging to the same sentence are concatenated together.
df_ner = df_ner.groupby(by="Sentence #").aggregate({
    "Word": lambda x: " ".join(x),
    "POS": lambda x: " ".join(x),
    "Tag": lambda x: " ".join(x)
})

df_ner.head()

,Word,POS,Tag
Sentence #,,,
Sentence: 1,Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
Sentence: 10,Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
Sentence: 100,Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...
Sentence: 1000,They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O
Sentence: 10000,U.N. relief coordinator Jan Egeland said Sunda...,"NNP NN NN NNP NNP VBD NNP , NNP , JJ CC JJ JJ ...",B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


###  create tokens and labels

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

text = "truebsdanimaltbj jhfbg@jadh(bgj) jklb, okay yes"
tokens = tokenizer(text, return_tensors='pt')  # Tokenize the text

decoded_tokens = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])  # Convert token IDs to tokens
print(decoded_tokens)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

['[CLS]', 'true', '##bs', '##dan', '##ima', '##lt', '##b', '##j', 'j', '##h', '##f', '##b', '##g', '@', 'j', '##ad', '##h', '(', 'b', '##g', '##j', ')', 'j', '##k', '##l', '##b', ',', 'okay', 'yes', '[SEP]']


In [ ]:
import re

#### Removing the special charaters
def remove_puntuation(text):
    pattern = "[^A-z0-9_\s]+"
    text = re.sub(pattern, "", text)
    return text


text = "truebsdanimaltbj jhfbg@jadh(bgj) jklb,  okay yes"
remove_puntuation(text)


'truebsdanimaltbj jhfbgjadhbgj jklb  okay yes'

### Since uncommon special words are converted to `##`
- It means the tokenized text lengths is not equal to label lengths we need to fix that either by
    1. removing `##` from input tokens
    2. Increasing the size of the labels which matches the input
    
### We will go with the method 2 to preprocess the text

In [ ]:
def prepare_sequences(text,tokens,labels):
    i = 0
    text = text.split()
    returned_labels = []
    formed_text = ""
    for t in tokens:

    #print(i,labels[i])
        if re.search("\[.*?\]",t):
            returned_labels.append('O')
        elif t == text[i]:
            returned_labels.append(labels[i])
            i += 1
        else:

            new_t = t.replace("#","")
            formed_text += new_t
            #print(formed_text)
            if text[i] == formed_text:
                #print(t,"here")
                returned_labels.append(labels[i])
                i += 1
                formed_text = ""
            else:
                returned_labels.append(labels[i])

    # else:
    #     returned_labels.append(labels[i])
    #print(i)
    return returned_labels

text = "truebsdanimaltbj Panindra jhfbg@jadh(bgj) jklb shf finedsaf"
labels = ['O', 'B', "O", "O",'O', 'A']
text = remove_puntuation(text)
print(text)
tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])

res = prepare_sequences(text, tokens, labels)
res = list(zip(tokens,res))
print(res)

truebsdanimaltbj Panindra jhfbgjadhbgj jklb shf finedsaf
[('[CLS]', 'O'), ('true', 'O'), ('##bs', 'O'), ('##dan', 'O'), ('##ima', 'O'), ('##lt', 'O'), ('##b', 'O'), ('##j', 'O'), ('Pan', 'B'), ('##ind', 'B'), ('##ra', 'B'), ('j', 'O'), ('##h', 'O'), ('##f', 'O'), ('##b', 'O'), ('##g', 'O'), ('##ja', 'O'), ('##dh', 'O'), ('##b', 'O'), ('##g', 'O'), ('##j', 'O'), ('j', 'O'), ('##k', 'O'), ('##l', 'O'), ('##b', 'O'), ('s', 'O'), ('##h', 'O'), ('##f', 'O'), ('fined', 'A'), ('##sa', 'A'), ('##f', 'A'), ('[SEP]', 'O')]


### The above example shows that the number of labels is equal to the number of tokens

### Let us preprocess the text

In [ ]:
df_ner['Word'] = df_ner['Word'].apply(lambda x: remove_puntuation(x))


### Let us get the number of classes in the dataset
- Number of classes means the number of Entities here

In [ ]:
df_ner.columns

Index(['Word', 'POS', 'Tag'], dtype='object')

In [ ]:
classes = set([c for labels in df_ner['Tag'].tolist() for c in labels.split()])
print(len(classes))
print(classes)

17
{'I-geo', 'B-nat', 'I-gpe', 'I-per', 'I-nat', 'O', 'B-org', 'I-eve', 'B-geo', 'I-tim', 'I-org', 'B-tim', 'B-eve', 'I-art', 'B-gpe', 'B-per', 'B-art'}


### We have 17 classes so the predictor of every token must have

#### Let us now look at the sequence length for the model

### Let us statistically look at the 75th Percentile


### 40 words will be chosen as sequence length for the modelling task

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", model_max_length=40,padding_side='left',truncation_side='right')


### Split the data into train and test sections
df_train, df_test = train_test_split(df_ner,random_state=3,train_size=0.75)

### Since we have 17 labels we have fixed the config of the model to `17` labels

### This is an important step to customize our requirement

In [ ]:

model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=17)


### BERT model has 2 Labels by default so we make the number of labels as 17
model.config.num_labels = 17




model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Pytorch Framework is used for training the model
- To train the model we need to create a torch DataLoader instance (custom DataLoader)

- We have to define an Optimizer `Adam` which optimezes/adjusts the learning rate for better convergence of the model which is momentum method

- We will define the number of epochs

- We are making use of the Token classifier model from transformers which means the loss function is `cross_entropy`. We do not have to specify it separately

- Training the model and evaluating it



### It is important to encode the labels into numbers

In [ ]:
mapper = {}

for i,c in enumerate(sorted(classes)):
    mapper[c] = i

print(mapper)

#### This function converts labels into numeric format i.e LabelEncoding
def process_labels(x):
    #x = x.split()
    res = [mapper[a] for a in x]
    return res


### applying mapper on labels
model.config.label2id = mapper
model.config.id2label = {v:k for k,v in mapper.items()}

{'B-art': 0, 'B-eve': 1, 'B-geo': 2, 'B-gpe': 3, 'B-nat': 4, 'B-org': 5, 'B-per': 6, 'B-tim': 7, 'I-art': 8, 'I-eve': 9, 'I-geo': 10, 'I-gpe': 11, 'I-nat': 12, 'I-org': 13, 'I-per': 14, 'I-tim': 15, 'O': 16}


### The below code is the class for custom Dataset
### This dataset will be sent to data loader

In [ ]:
#### Below code creates the dataloader

from torch.utils.data import Dataset, DataLoader

class NerDataset(Dataset):

    def __init__(self, df, text_col, label_col,
                 tokenizer=tokenizer, seq_len=40):
        self.df = df
        self.text_col = text_col
        self.label_col = label_col
        self.seq_len = seq_len
        self.tokenizer = tokenizer

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self,idx):
        text = self.df[self.text_col].iloc[idx]
        tokens = self.tokenizer(text, max_length=self.seq_len,
                                padding='max_length', truncation=True,
                               return_tensors='pt')
        ids = list(tokens['input_ids'].numpy().reshape(-1))

        words = self.tokenizer.convert_ids_to_tokens(ids)
        #print(words)
        ### Getting the labels
        labels = self.df[self.label_col].iloc[idx].split()
        ### Fixing the length of the labels
        labels = prepare_sequences(text,words,labels)
        labels = process_labels(labels)
        labels = torch.tensor(labels)
        #print(list(zip(words,labels)))
        ### If GPU is available then move the Data to GPU:
        if torch.cuda.is_available():
            tokens['input_ids'] = tokens['input_ids'].to('cuda')
            tokens['token_type_ids'] = tokens['token_type_ids'].to('cuda')
            tokens['attention_mask'] = tokens['attention_mask'].to('cuda')
            labels = labels.to('cuda')
        return tokens, labels



In [ ]:
train_ds = NerDataset(df=df_train,text_col='Word',label_col='Tag')
test_ds = NerDataset(df=df_test,text_col='Word',label_col='Tag')


#### Now creating DataLoader
train_data = DataLoader(train_ds, batch_size=64,shuffle=True)
test_data = DataLoader(test_ds, batch_size=64,shuffle=True)


### Let us train the model with some training functions

In [ ]:
epochs = 5
if torch.cuda.is_available():
    model = model.to("cuda")


from torch.optim import Adam
optimizer = Adam(lr=2e-6, params=model.parameters())



def get_steps(total, batch_size):
    if total % batch_size == 0:
        return total // batch_size
    else:
        return total // batch_size + 1


train_steps  = get_steps(df_train.shape[0],64)
val_steps  = get_steps(df_test.shape[0],64)
### Train step
def train(dataset,model):
    model.train()

    running_loss = 0
    i = 1
    running_accuracy = 0
    for d,l in dataset:
        d['input_ids'] = d['input_ids'][:,0,:]
        d['token_type_ids'] = d['token_type_ids'][:,0,:]
        d['attention_mask'] = d['attention_mask'][:,0,:]
        optimizer.zero_grad()
        out = model(**d,labels=l)
        loss = out.loss
        running_loss += loss.item()
        #print(out.logits.shape)

        y = out.logits.argmax(axis=2).cpu().detach().numpy()
        l = l.cpu().detach().numpy()

        running_accuracy += np.sum(y == l)/(l.shape[0]*l.shape[1])
        #print(np.sum(y == l)/l.shape[0]*l.shape[1])
        loss.backward()
        optimizer.step()
        eqs = 70*i//train_steps
        sp = 70*(1-i//train_steps)
        if i < train_steps:
            print("<" + "="*eqs + " "*sp + ">",
             f"loss={running_loss/i} , accuracy={running_accuracy/i}",end='\r')
        else:
            print("<" + "="*eqs + " "*sp + ">",
             f"loss={running_loss/i} , accuracy={running_accuracy/i}",end='\n')
        i+=1



def evaluate(dataset,model):
    model.eval()

    running_loss = 0
    i = 1
    running_accuracy = 0
    for d,l in dataset:
        #optimizer.zero_grad()
        d['input_ids'] = d['input_ids'][:,0,:]
        d['token_type_ids'] = d['token_type_ids'][:,0,:]
        d['attention_mask'] = d['attention_mask'][:,0,:]
        out = model(**d,labels=l)
        loss = out.loss
        running_loss += loss.item()

        y = out.logits.argmax(axis=2).cpu().detach().numpy()
        l = l.cpu().detach().numpy()

        running_accuracy += np.sum(y == l)/(l.shape[0]*l.shape[1])
        #loss.backward()
        #optimizer.step()
        eqs = 70*i//val_steps
        sp = 70*(1-i//val_steps)

        print("<" + "="*eqs + " "*sp + ">",
             f"val_loss={running_loss/i} , val_accuracy={running_accuracy/i}",
             end='\r')

        i+=1



In [ ]:
for e in range(epochs):
    print(print(f"Epoch {e+1}/{epochs}"), end=" ")
    train( train_data, model)


Epoch 1/5
<======================================================================> loss=1.3541054675976436 , accuracy=0.7724218750000001
Epoch 2/5


In [ ]:
evaluate(test_data, model)

In [ ]:
import torch
torch.cuda.empty_cache()

### Let us predict on realtime validation data

In [ ]:
### Loading a Sample dataset
sample_df = df_test.sample(20, random_state=4,replace=True)

texts = sample_df['Word'].tolist()

### Tokenize the data
tokens = tokenizer(texts, return_tensors='pt', max_length=40,
                   padding='max_length', truncation=True)
if torch.cuda.is_available():
    tokens['input_ids'] = tokens['input_ids'].to('cuda')
    tokens['token_type_ids'] = tokens['token_type_ids'].to('cuda')
    tokens['attention_mask'] = tokens['attention_mask'].to('cuda')


seq = pd.Series(list(tokens['input_ids'].cpu().numpy()))
#token_text = tokenizer.convert_ids_to_tokens(list(tokens['input_ids'].cpu().numpy()))
seq = seq.apply(lambda x: list(x))
seq = seq.apply(lambda x: tokenizer.convert_ids_to_tokens(x))
seq

### Stored Inputs as a DataFrame

#### Let us process the outputs

In [ ]:
predictions = model(**tokens).logits.argmax(axis=2)
predictions

### Let us now decode the predicted labels as `strings`

In [ ]:
mapper

In [ ]:

preds = predictions.cpu().numpy()

### We will use the mapper ditionary which was created in the previous cells for the step
def covert_preds_to_str(val, mapper=mapper):
    for k,v in mapper.items():
        if val == v:
            label = k
    return label


fun = np.vectorize(covert_preds_to_str)
preds_text = fun(preds)
preds_text = pd.Series(list(preds_text))
preds_text




### Now Compining both of them

In [ ]:
df_res = pd.concat([seq, preds_text], axis=1)
df_res.columns = ['Tokens', 'Labels']
df_res['Combined_Result'] = df_res.apply(lambda x: list(zip(*x)), axis=1)
df_res

,Tokens,Labels,Combined_Result
0,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
1,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
2,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
3,"[[PAD], [PAD], [PAD], [PAD], [CLS], The, White...","[O, O, O, O, O, O, B-org, I-org, O, O, O, O, O...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
4,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
5,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, B-org, B-per, I-per, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
6,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
7,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
8,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
9,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."


### So the Ultimate step is to combine with the test Dataset

In [ ]:
df_res.index = sample_df.index
df_res = pd.concat([sample_df, df_res], axis=1)
df_res

,Word,POS,Tag,Tokens,Labels,Combined_Result
Sentence #,,,,,,
Sentence: 3593,Tehran denies it is trying to build a nuclear ...,NNP VBZ PRP VBZ VBG TO VB DT JJ NN .,B-geo O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 4389,The plane went down in water some 7000 metersd...,DT NN VBD RB IN NN DT CD NN IN DT JJ NN NN .,O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 2948,The group also left a note saying the killings...,DT NN RB VBD DT NN VBG DT NNS VBD NN IN WP PRP...,O O O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 6222,The White House said the president was briefed...,DT NNP NNP VBD DT NN VBD VBN RB IN JJ NNS IN D...,O B-org I-org O O O O O O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [CLS], The, White...","[O, O, O, O, O, O, B-org, I-org, O, O, O, O, O...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 6158,Researchers say US consumers have a different ...,NNS VBP NNP NNS VBP DT JJ NN IN DT NN .,O O B-geo O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 1454,UN SecretaryGeneral Kofi Annan has officially ...,NNP NNP NNP NNP VBZ RB VBN DT NN IN CD JJ NNS ...,B-org I-org B-per I-per O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, B-org, B-per, I-per, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 2625,A massive 79 quake hit Sichuan province in 200...,"DT JJ CD NN VBD NNP NN IN CD , VBG RB TO CD NNS .",O O O O O B-geo O O B-tim O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 4496,Kennedy also said he was grateful for the pray...,NNP RB VBD PRP VBD JJ IN DT NNS CC JJ NNS PRP ...,B-per O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."
Sentence: 6078,Approval from 56 member nations would be neede...,NNP IN CD NN NNS MD VB VBN TO VB DT NN .,O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O..."


### Let us make the final result without `[PAD]` tokens and `[CLS]` tokens also we will remove the tokens with `#`


In [ ]:
word_list = df_res['Word'].apply(lambda x: x.split()).tolist()
fp = df_res['Final_Preds'].tolist()

clean_out = []
for i in range(len(word_list)):
    clean_out.append(list(zip(word_list[i], fp[i])))

df_res['Clean_Output'] = clean_out

df_res


,Word,POS,Tag,Tokens,Labels,Combined_Result,Final_Preds,Clean_Output
Sentence #,,,,,,,,
Sentence: 3593,Tehran denies it is trying to build a nuclear ...,NNP VBZ PRP VBZ VBG TO VB DT JJ NN .,B-geo O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[B-geo, O, O, O, O, O, O, O, O, O]","[(Tehran, B-geo), (denies, O), (it, O), (is, O..."
Sentence: 4389,The plane went down in water some 7000 metersd...,DT NN VBD RB IN NN DT CD NN IN DT JJ NN NN .,O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[(The, O), (plane, O), (went, O), (down, O), (..."
Sentence: 2948,The group also left a note saying the killings...,DT NN RB VBD DT NN VBG DT NNS VBD NN IN WP PRP...,O O O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(The, O), (group, O), (also, O), (left, O), (..."
Sentence: 6222,The White House said the president was briefed...,DT NNP NNP VBD DT NN VBD VBN RB IN JJ NNS IN D...,O B-org I-org O O O O O O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [CLS], The, White...","[O, O, O, O, O, O, B-org, I-org, O, O, O, O, O...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, B-org, I-org, O, O, O, O, O, O, O, O, O, O...","[(The, O), (White, B-org), (House, I-org), (sa..."
Sentence: 6158,Researchers say US consumers have a different ...,NNS VBP NNP NNS VBP DT JJ NN IN DT NN .,O O B-geo O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, B-geo, O, O, O, O, O, O, O, O]","[(Researchers, O), (say, O), (US, B-geo), (con..."
Sentence: 1454,UN SecretaryGeneral Kofi Annan has officially ...,NNP NNP NNP NNP VBZ RB VBN DT NN IN CD JJ NNS ...,B-org I-org B-per I-per O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, B-org, B-per, I-per, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[B-org, B-per, I-per, I-per, O, O, O, O, O, O,...","[(UN, B-org), (SecretaryGeneral, B-per), (Kofi..."
Sentence: 2625,A massive 79 quake hit Sichuan province in 200...,"DT JJ CD NN VBD NNP NN IN CD , VBG RB TO CD NNS .",O O O O O B-geo O O B-tim O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, B-geo, O, O, O, O, O, O, O, O]","[(A, O), (massive, O), (79, O), (quake, O), (h..."
Sentence: 4496,Kennedy also said he was grateful for the pray...,NNP RB VBD PRP VBD JJ IN DT NNS CC JJ NNS PRP ...,B-per O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[(Kennedy, O), (also, O), (said, O), (he, O), ..."
Sentence: 6078,Approval from 56 member nations would be neede...,NNP IN CD NN NNS MD VB VBN TO VB DT NN .,O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O]","[(Approval, O), (from, O), (56, O), (member, O..."


In [ ]:
def remove_special_tokens(x):
    res = []
    for token, label in x:
        if re.search("\[.*\]",token) is None and '#' not in token:
            res.append(label)
    return res



df_res['Final_Preds'] = df_res['Combined_Result'].apply(lambda x: remove_special_tokens(x))
df_res

,Word,POS,Tag,Tokens,Labels,Combined_Result,Final_Preds
Sentence #,,,,,,,
Sentence: 3593,Tehran denies it is trying to build a nuclear ...,NNP VBZ PRP VBZ VBG TO VB DT JJ NN .,B-geo O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[B-geo, O, O, O, O, O, O, O, O, O]"
Sentence: 4389,The plane went down in water some 7000 metersd...,DT NN VBD RB IN NN DT CD NN IN DT JJ NN NN .,O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
Sentence: 2948,The group also left a note saying the killings...,DT NN RB VBD DT NN VBG DT NNS VBD NN IN WP PRP...,O O O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
Sentence: 6222,The White House said the president was briefed...,DT NNP NNP VBD DT NN VBD VBN RB IN JJ NNS IN D...,O B-org I-org O O O O O O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [CLS], The, White...","[O, O, O, O, O, O, B-org, I-org, O, O, O, O, O...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, B-org, I-org, O, O, O, O, O, O, O, O, O, O..."
Sentence: 6158,Researchers say US consumers have a different ...,NNS VBP NNP NNS VBP DT JJ NN IN DT NN .,O O B-geo O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, B-geo, O, O, O, O, O, O, O, O]"
Sentence: 1454,UN SecretaryGeneral Kofi Annan has officially ...,NNP NNP NNP NNP VBZ RB VBN DT NN IN CD JJ NNS ...,B-org I-org B-per I-per O O O O O O O O O O O ...,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, B-org, B-per, I-per, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[B-org, B-per, I-per, I-per, O, O, O, O, O, O,..."
Sentence: 2625,A massive 79 quake hit Sichuan province in 200...,"DT JJ CD NN VBD NNP NN IN CD , VBG RB TO CD NNS .",O O O O O B-geo O O B-tim O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, B-geo, O, O, O, O, O, O, O, O]"
Sentence: 4496,Kennedy also said he was grateful for the pray...,NNP RB VBD PRP VBD JJ IN DT NNS CC JJ NNS PRP ...,B-per O O O O O O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
Sentence: 6078,Approval from 56 member nations would be neede...,NNP IN CD NN NNS MD VB VBN TO VB DT NN .,O O O O O O O O O O O O O,"[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[([PAD], O), ([PAD], O), ([PAD], O), ([PAD], O...","[O, O, O, O, O, O, O, O, O, O, O, O]"


### Saving the output and the model

In [ ]:
df_res.to_csv("Output.csv")
model.save_pretrained("Pytorch-Tuned-NER-BERT")

### Lets have a glimpse of the output

In [ ]:
x = df_res['Clean_Output'].tolist()

for i in x:
    print(i)
    print("\n"*3)

[('Tehran', 'B-geo'), ('denies', 'O'), ('it', 'O'), ('is', 'O'), ('trying', 'O'), ('to', 'O'), ('build', 'O'), ('a', 'O'), ('nuclear', 'O'), ('weapon', 'O')]




[('The', 'O'), ('plane', 'O'), ('went', 'O'), ('down', 'O'), ('in', 'O'), ('water', 'O'), ('some', 'O'), ('7000', 'O'), ('metersdeep', 'O'), ('above', 'O'), ('a', 'O'), ('mountainous', 'O'), ('ocean', 'O'), ('floor', 'O')]




[('The', 'O'), ('group', 'O'), ('also', 'O'), ('left', 'O'), ('a', 'O'), ('note', 'O'), ('saying', 'O'), ('the', 'O'), ('killings', 'O'), ('were', 'O'), ('part', 'O'), ('of', 'O'), ('what', 'O'), ('it', 'O'), ('called', 'O'), ('divine', 'O'), ('justice', 'O')]




[('The', 'O'), ('White', 'B-org'), ('House', 'I-org'), ('said', 'O'), ('the', 'O'), ('president', 'O'), ('was', 'O'), ('briefed', 'O'), ('earlier', 'O'), ('by', 'O'), ('senior', 'O'), ('aides', 'O'), ('about', 'O'), ('the', 'O'), ('incident', 'O'), ('and', 'O'), ('strongly', 'O'), ('supports', 'O'), ('what', 'O'), ('it', 'O'), ('said', 'O'), ('

In [ ]:
import numpy as np

cleaned_true_labels = np.array(labels).squeeze().tolist()
cleaned_preds = np.array(preds).squeeze().tolist()

print(classification_report(labels, cleaned_preds, target_names=['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']))

ValueError: ignored

In [ ]:
from transformers import BertForTokenClassification, BertTokenizer
import torch
import torch.nn.functional as F

# Load the previously saved model - replace "Pytorch-Tuned-NER-BERT" with the correct model name
loaded_model = BertForTokenClassification.from_pretrained("Pytorch-Tuned-NER-BERT")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", model_max_length=40, padding_side='left', truncation_side='right')

# Example user input
user_text = "London is a great place to live and Iraq is having protests going on. India and Turkish"

if user_text.strip() == "":
    print("Input text is empty. Provide a non-empty text for tokenization and prediction.")
else:
    # Tokenize user input using the loaded tokenizer
    tokens = tokenizer.encode_plus(user_text, add_special_tokens=True, return_tensors='pt')

    # Ensure correct input shape and type (adjust sequence length if needed)
    max_seq_length = 40  # Using the max length defined in the tokenizer
    if tokens['input_ids'].size(1) > max_seq_length:
        tokens['input_ids'] = tokens['input_ids'][:, :max_seq_length]

    # Model prediction
    with torch.no_grad():
        outputs = loaded_model(**tokens)
        logits = outputs.logits  # Get the logits directly

    # Apply softmax to obtain probabilities
    probs = F.softmax(logits, dim=2)

    # Get the predicted labels based on probabilities
    predicted_labels = torch.argmax(probs, dim=2)

    # Define the label mapping based on the model's output indices
    label_map = {0: 'B-art', 1: 'B-eve', 2: 'B-geo', 3: 'B-gpe', 4: 'B-nat', 5: 'B-org', 6: 'B-per',
                 7: 'B-tim', 8: 'I-art', 9: 'I-eve', 10: 'I-geo', 11: 'I-gpe', 12: 'I-nat', 13: 'I-org',
                 14: 'I-per', 15: 'I-tim', 16: 'O'}

    # Map predicted label indices to labels using the defined function
    predicted_labels_mapped = [label_map[idx.item()] for idx in predicted_labels[0][1:-1]]  # Exclude first and last labels

    # Display the mapped labels
    print(user_text)
    print("Predicted Labels:", predicted_labels_mapped)


London is a great place to live and Iraq is having protests going on. India and Turkish
Predicted Labels: ['B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'B-geo']
